# 02. Conference Top Authors By Static Rank   

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import *
from CreateFunctions_mag import *
import pyspark.sql.functions as sf

In [ ]:
rootpath = 'wasbs://mag-2018-09-27@magtrainingsource.blob.core.windows.net/mag/'
outputDir = '/output/jiaxin/pyspark/'
targetConf = 'WWW'
n_top = 20

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
# First find the conference series id by matching conference short name
ConferenceSeries = ConferenceSeries(rootpath, spark)

targetConferenceSeriesId = ConferenceSeries.where(sf.col("NormalizedName").isin(targetConf)) \
.select("ConferenceSeriesId")


# Get all conference papers by conference series Id
Papers = Papers(rootpath, spark)

Papers = Papers .join(ConferenceSeries, "ConferenceSeriesId", 'inner') \
.selectExpr("PaperId", "Rank * -0.001 as LogProbRank") # Rank is stored in the graph as LogProbRank*-1000


# Get all [conference paper] -> [conference author] relationships
PaperAuthorAffiliations = PaperAuthorAffiliations(rootpath, spark)

Conf_Authors = PaperAuthorAffiliations.join(Papers, "PaperId", 'inner') \
.select("AuthorId", "LogProbRank")


# Aggregate static rank and publication for each author
Authors = Authors(rootpath, spark)

Authors = Authors.join(Conf_Authors, "AuthorId", 'inner') \
.select("DisplayName", "LogProbRank")

AuthorRank = Authors.groupby("DisplayName") \
.agg(sf.sum(sf.exp("LogProbRank")).alias("AuthorProbRank")) \
.select("DisplayName", "AuthorProbRank")

PublicationCounts = Authors.groupby("DisplayName") \
.count() \
.selectExpr("DisplayName", "count as PublicationCount")

AuthorProbRank  = AuthorRank.join(PublicationCounts, "DisplayName", 'inner') \
.sort(sf.desc("AuthorProbRank")) \
.limit(n_top)


# Create two ranks based on different methods to display on the x and y axis for comparison
conferenceAuthorRanks = AuthorProbRank.select("DisplayName", "PublicationCount", 
                                        sf.rank().over(Window.orderBy("AuthorProbRank")).alias("AuthorRank"), 
                                        sf.rank().over(Window.orderBy(sf.col("AuthorProbRank") / sf.col("PublicationCount"))).alias("AuthorNormalizedRank"))
# Consider average quality of paper instead of sum


conferenceAuthorRanks.write.csv(outputDir + 'conferenceAuthorRanks.csv', mode='overwrite', header='true')

In [ ]:
sc.stop()